In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
import requests
from IPython.display import display, Markdown
import google.generativeai as genai

# ====================== CONFIGURATION ======================
# Initialize Gemini
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"  # Replace with your actual key
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel('gemini-pro')

# Together API Key
TOGETHER_API_KEY = "tgp_v1_S8njNTMyg7Knr0Gb9UWjLpGOhtcLEpZSM14ywKDAinc"  # Replace with your key

# ====================== TRANSLATION FUNCTIONS ======================
def translate_with_gemini(text, target_language='Arabic'):
    """Translate text using Gemini"""
    try:
        if not text or not isinstance(text, str):
            return text
            
        response = gemini_model.generate_content(
            f"Translate this to {target_language} exactly without adding any extra text: {text}"
        )
        return response.text
    except Exception as e:
        print(f"Gemini translation error: {e}")
        return text

# ====================== DATA PROCESSING ======================
try:
    file_path = "C:/Users/ASUS/Documents/GitHub/github.com/recommendation-system/Dataset/cleaned_dataset.csv"
    df = pd.read_csv(file_path)
    df["Original_Area"] = df["Area"]
    df["Original_Price"] = df["Price"]
except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    raise

# Preprocess the data
features = ["Area", "Price"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# Apply KMeans clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df["Cluster"] = kmeans.fit_predict(X_scaled)

# Function to make recommendations based on KMeans clustering
def recommend_properties_kmeans(price_input, area_input, top_n=5):
    input_df = pd.DataFrame([[area_input, price_input]], columns=["Area", "Price"])
    input_scaled = scaler.transform(input_df)
    input_cluster = kmeans.predict(input_scaled)[0]

    # Filter cluster by constraints
    cluster_data = df[df["Cluster"] == input_cluster].copy()
    cluster_data = cluster_data[
        (cluster_data["Original_Area"] >= area_input) &
        (cluster_data["Original_Price"] <= price_input)
    ]

    if cluster_data.empty:
        return pd.DataFrame()

    cluster_scaled = scaler.transform(cluster_data[["Area", "Price"]])
    distances = euclidean_distances(input_scaled, cluster_scaled)[0]

    cluster_data["Distance"] = distances
    recommended = cluster_data.sort_values("Distance").head(top_n)

    recommended = recommended.rename(columns={
        "Property Type": "Property Type",
        "Location": "Location",
        "District": "District",
        "Bedrooms": "Bedrooms",
        "Bathrooms": "Bathrooms",
        "Original_Area": "Area",
        "Original_Price": "Price",
        "Agency_Name": "Agency"
    })

    headers = ["Property Type", "Location", "District", "Bedrooms", "Bathrooms", "Area", "Price", "Agency"]
    return recommended[headers]

# ====================== TEMPLATES ======================
def template_informative(properties):
    """Professional Arabic property descriptions"""
    explanation = translate_with_gemini(
        "You are a professional real estate assistant.\n"
        "Display details in Arabic only.\n\n"
        "Use only the following field labels:\n"
        "Property Type, Location, District, Bedrooms, Bathrooms, Area, Price, Agency\n\n"
        "Do not translate or change any of the values, especially Arabic values like: apartment, Al-Ahmadi Real Estate Office, East - Cordoba.\n"
        "Keep them as they are.\n\n"
        "Display each property in the following style:\n"
        "- Property Type: (value)\n"
        "- Location: (value)\n"
        "- District: (value)\n"
        "- Bedrooms: (value)\n"
        "- Bathrooms: (value)\n"
        "- Area: (value) square meters\n"
        "- Price: (value) SAR\n"
        "- Agency: (value)\n\n"
        "After displaying each property, add a short paragraph in Arabic professionally describing the property, explaining who it is suitable for and what distinguishes it.\n\n"
        "Recommended properties:\n\n"
    )

    for i, prop in enumerate(properties):
        explanation += (
            f"{i+1}.\n"
            f"{translate_with_gemini('- Property Type:')} {prop['Property Type']}\n"
            f"{translate_with_gemini('- Location:')} {prop['Location']}\n"
            f"{translate_with_gemini('- District:')} {prop['District']}\n"
            f"{translate_with_gemini('- Bedrooms:')} {prop['Bedrooms']}\n"
            f"{translate_with_gemini('- Bathrooms:')} {prop['Bathrooms']}\n"
            f"{translate_with_gemini('- Area:')} {prop['Area']} {translate_with_gemini('square meters')}\n"
            f"{translate_with_gemini('- Price:')} {prop['Price']} {translate_with_gemini('SAR')}\n"
            f"{translate_with_gemini('- Agency:')} {prop['Agency']}\n\n"
        )

    return explanation

def template_friendly_descriptive(properties):
    """Generate friendly Arabic descriptions"""
    explanations = []
    
    welcome_msg = translate_with_gemini("Welcome to our exclusive real estate offers!")
    explanations.append(f"{welcome_msg}\n\n")
    
    for i, prop in enumerate(properties):
        prompt = (
            f"Write a friendly real estate description in English for:\n"
            f"Type: {prop['Property Type']}\nLocation: {prop['Location']}\n"
            f"District: {prop['District']}\nBedrooms: {prop['Bedrooms']}\n"
            f"Bathrooms: {prop['Bathrooms']}\nArea: {prop['Area']} sqm\n"
            f"Price: {prop['Price']} SAR\nAgency: {prop['Agency']}\n\n"
            "Instructions:\n1. Start with a welcoming sentence\n"
            "2. Describe key features naturally\n3. Mention price convincingly\n"
            "4. Help visualize living there\n5. End with call to action\n"
            "6. Keep it 6-8 sentences, conversational but professional"
        )
        
        try:
            english_description = call_llama(prompt)
            if not english_description:
                raise ValueError("Empty description from LLM")
                
            arabic_description = translate_with_gemini(english_description)
            
            property_header = translate_with_gemini(f"Property {i+1}:")
            explanations.append(f"{property_header}\n{arabic_description.strip()}\n")
            
        except Exception as e:
            print(f"Error generating description for property {i+1}: {e}")
            fallback_text = (
                f"{translate_with_gemini('Property')} {i+1}:\n"
                f"{translate_with_gemini('Type:')} {prop['Property Type']}\n"
                f"{translate_with_gemini('Location:')} {prop['Location']}\n"
                f"{translate_with_gemini('District:')} {prop['District']}\n"
                f"{translate_with_gemini('Bedrooms:')} {prop['Bedrooms']}\n"
                f"{translate_with_gemini('Bathrooms:')} {prop['Bathrooms']}\n"
                f"{translate_with_gemini('Area:')} {prop['Area']} {translate_with_gemini('sqm')}\n"
                f"{translate_with_gemini('Price:')} {prop['Price']} {translate_with_gemini('SAR')}\n"
                f"{translate_with_gemini('Agency:')} {prop['Agency']}\n\n"
            )
            explanations.append(fallback_text)
    
    closing_msg = translate_with_gemini(
        "\nWe hope you find what you're looking for among these offers!\n"
        "Feel free to contact us to arrange a visit or for more details.\n"
        "We're here to help you find your dream home."
    )
    explanations.append(closing_msg)
    
    return "\n".join(explanations)

# ====================== TOGETHER API ======================
def call_llama(prompt, model="mistralai/Mixtral-8x7B-Instruct-v0.1", temperature=0.6, top_p=0.85):
    if not TOGETHER_API_KEY:
        raise ValueError("API key not found.")

    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a helpful real estate assistant."},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 7000,
        "temperature": temperature,
        "top_p": top_p,
        "repetition_penalty": 1.1,
    }

    try:
        response = requests.post(
            "https://api.together.xyz/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Other error occurred: {err}")

# ====================== MAIN EXECUTION ======================
if __name__ == "__main__":
    recommended = recommend_properties_kmeans(price_input=2000000, area_input=300)
    if recommended.empty:
        print("No properties found based on your input.")
    else:
        # Generate and display both templates
        info_output = template_informative(recommended)
        friendly_output = template_friendly_descriptive(recommended)
        
        display(Markdown("### Template 1: Informative Guide"))
        display(Markdown(info_output.strip()))
        
        display(Markdown("### Template 2: Casual Guide"))
        display(Markdown(friendly_output.strip()))

Translation init error: 403 POST https://translation.googleapis.com/language/translate/v2?prettyPrint=false: Cloud Translation API has not been used in project 304270238783 before or it is disabled. Enable it by visiting htt


### Professional Listing

أنت مساعد عقاري محترف.
اعرض التفاصيل باللغة العربية فقط.

العقارات المقترحة:

619.
- نوع العقار: فیلا
- الموقع: غرب
- الحي: المهدية
- الغرف: 5
- دورات المياه: 5
- المساحة: 312 متر مربع
- السعر: 1800000 ريال
- الوكالة: بيت النائف للتطوير العقاري

233.
- نوع العقار: فیلا
- الموقع: شرق
- الحي: الفيحاء
- الغرف: 9
- دورات المياه: 6
- المساحة: 334 متر مربع
- السعر: 1800000 ريال
- الوكالة: مساكن ركاز للعقارات

765.
- نوع العقار: دور
- الموقع: شرق
- الحي: المونسية
- الغرف: 3
- دورات المياه: 5
- المساحة: 301 متر مربع
- السعر: 1700000 ريال
- الوكالة: مؤسسة وسام المساكن العقارية

462.
- نوع العقار: عمارة
- الموقع: وسط
- الحي: منفوحة
- الغرف: 11
- دورات المياه: 7
- المساحة: 316 متر مربع
- السعر: 1700000 ريال
- الوكالة: مكتب اضداد للعقارات

135.
- نوع العقار: فیلا
- الموقع: شرق
- الحي: الجنادرية
- الغرف: 6
- دورات المياه: 6
- المساحة: 359 متر مربع
- السعر: 1800000 ريال
- الوكالة: شركة مدى الاعصار العقارية



### Friendly Descriptions

Welcome to our real estate offers!

Property 619:
 Welcome to this stunning 5-bedroom, 5-bathroom villa, located in the prestigious Al Mehdiya district in the west of the city. Spanning 312 sqm, this property offers spacious living areas filled with natural light and modern finishes. The villa boasts a sleek and modern kitchen, perfect for entertaining. With a private outdoor space, you can enjoy the beautiful weather in the privacy of your own home. This property is priced at 1,800,000 SAR and is presented by the reputable real estate agency, Bayt Al Naef for Real Estate Development. Don't miss out on the opportunity to make this dream villa your new home.

Property 233:
 Welcome to this stunning 9-bedroom vill (فیلا) in the heart of the vibrant district of Al Fihaa, East (شرق) region. Spanning 334 sqm, this spacious property offers a perfect blend of comfort and luxury with 6 well-appointed bathrooms. Imagine waking up to breathtaking views and enjoying a cup of coffee in your expansive living area. With top-notch amenities, this villa offers unparalleled privacy, security, and convenience, making it an ideal choice for families or anyone seeking a serene and sophisticated lifestyle. This property is listed at an attractive price of 1,800,000 SAR by Rekaz Real Estate (مساكن ركاز للعقارات), a trusted name in the industry. Don't miss out on this opportunity to own your dream home in one of the most desirable locations in town!

Property 765:
 Welcome to this stunning 3-bedroom villa, located in the heart of the esteemed district of Al Mowansiyah, in the vibrant East region of the city. Spanning 301 sqm, this spacious property boasts 5 luxurious bathrooms, providing the ultimate in comfort and privacy for you and your family. With its modern design, high-quality finishes, and top-of-the-line amenities, this villa is sure to exceed your expectations. Enjoy the convenience of being close to all the best that the city has to offer, while still enjoying the peace and tranquility of a private residence. This exceptional property is available for just 1,700,000 SAR through Wasam Real Estate Agency. Don't miss out on the opportunity to make this dream villa your own.

Property 462:
 Welcome to this stunning 11-bedroom villa, located in the heart of Manfuha district! Spanning 316 sqm, this عمارة offers a spacious and comfortable living experience. With 7 modern bathrooms, there's plenty of room for the whole family. The location is prime, situated in the center of the district, making it convenient for all your needs. This property is listed for sale at 1,700,000 SAR with Maktoub Al-Etihad Real Estate Agency. Don't miss out on the opportunity to make this villa your new home!

Property 135:
 Discover luxury living in the heart of Janadriyah with this stunning 6-bedroom, 6-bathroom villa! Spanning 359 sqm, this property offers ample space for you and your family to grow. Each bedroom is thoughtfully designed with attached bathrooms for maximum privacy and convenience. The villa is located in the desirable district of Sharq, providing easy access to top-rated schools, shopping centers, and popular attractions. With a price of 1,800,000 SAR, this is an opportunity you won't want to miss. Contact Mady Al Awsat Real Estate Agency today to schedule a viewing!


Contact us for viewings or more information!